In [28]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/21 14:28
# @Author  : Wang Yujia
# @File    : data_cal_np.ipynb
# @Description : 1. 计算N和P 2. 设置threshold

# 0. what for
1. 计算N和P，从`data_extract_for_asc_symmetry_2.ipynb`提取出来的
    - 计算P的方法：**method-2**，`(price-0)/bidincrement`
2. 计算完后，样本数小于`threshold`的settings不予考虑
3. Output：2个csv
    - `datawithnp_2.csv`：去除了重复行，但是未没经过threshold筛选的data
    - `datawithnp_2_selected.csv`去除了重复行，且经过threshold筛选
4. **检查逻辑的2个思路**：
    - 对于同一个unique setting, 是不是 sum(['cnt_n_1'])==['cnt_uniq'].
    - 对于同一个unique setting, 是不是['cnt_n_1']==2对应的['P']是['cnt_n_1']==1对应的['P']的 2倍
    - 逻辑检查 see 2.2.4
# 1. Preparations
## 1.1 全局设置

In [53]:
# outcomes dataset
outcomes_path = "../data/outcomes_cleaned.csv"

# 计算完n和p后，可以使用的dataset被output在
output_path = "../data/datawithnp_2.csv"
# 经过threshold后，得到的dataset
data_selected_path = "../data/datawithnp_2_selected.csv"

# 衡量一场auction是否unique的标志
unique_setting = ['id', 'bidincrement', 'bidfee','retail']

threshold = 16

import numpy as np
import pandas as pd

## 1.2 读取data

In [54]:
outcomes = pd.read_csv(outcomes_path, encoding="utf-8")

# 2. 计算N和P
## 2.1 计算N
1. 这里计算N是 **“方法二”** ：在`outcomes.tsv`中通过(price-0)/bidincrement来计算

In [55]:
data_withn = outcomes.copy()
data_withn.loc[:,'n_2'] = outcomes.loc[:,'price'] / (outcomes.loc[:,'bidincrement'])
data_withn['n_2'] = data_withn['n_2'].astype(int)                      # 把n变成int

## 2.2 计算p
1. unique setting一样的auction认为是同一个
2. `P = cnt_n_2 / cnt_uniq`
3. `cnt_uniq = ['n_2'].nunique()` when 'unique setting' is the same

In [56]:
# 2.2.1 each unique setting对应了'cnt_uniq'场auction: data_withn_cnt
# 注意'cnt_uniq'本来并不需要出现在最后的data中，但是后面用threshold会联动一下
# 最后的data中保留了这一项，方便计算likelihood

data_grouped_tmp = data_withn.groupby(unique_setting,as_index=False)
tmp = pd.DataFrame(data_grouped_tmp.size())
# 现在是3666个auction uniq settings

In [57]:
data_withn_cnt = pd.merge(data_withn, tmp, on=unique_setting, how="left")
data_withn_cnt.rename(columns={'size': 'cnt_uniq'}, inplace=True)
data_withn_cnt.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,placedbids,freebids,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,id,n_2,cnt_uniq
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,0.15,0.75,Racer11,26,0,19:52 PDT 09-16-2008,0,0,0,0,0,89,69
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,0.15,0.75,Cemo23,65,0,11:17 PDT 08-28-2008,0,0,0,0,1,498,60
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,0.15,0.75,Jacobsonnich,94,0,22:52 PDT 11-07-2008,0,1,0,0,1,554,60
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,10,0,22:02 PDT 08-23-2008,0,0,0,0,0,131,69
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,80,0,14:23 PDT 08-24-2008,0,0,0,0,0,314,69


- P = cnt_n / cnt_uniq,上面已经算了' cnt_uniq'，下面需要算cnt_n

In [58]:
# 2.2.2 计算cnt_n_2，并添加到data_withn的一列: data_withn_cnt_n1
        # cnt_n_2表示某个setting下的n_1某数值出现了几次/ Example: cnt_n=2表示在某个setting下，有2场拍卖持续了n_1轮
features_cnt_n_2 = unique_setting+['n_2']
data_grouped_tmp = data_withn.groupby(features_cnt_n_2,as_index=False)
tmp = pd.DataFrame(data_grouped_tmp.size())

data_withn_cnt_n2 = pd.merge(data_withn_cnt, tmp, on=features_cnt_n_2, how="left")
data_withn_cnt_n2.rename(columns={'size': 'cnt_n_2'}, inplace=True)
data_withn_cnt_n2.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,...,freebids,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,id,n_2,cnt_uniq,cnt_n_2
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,0.15,0.75,Racer11,...,0,19:52 PDT 09-16-2008,0,0,0,0,0,89,69,1
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,0.15,0.75,Cemo23,...,0,11:17 PDT 08-28-2008,0,0,0,0,1,498,60,1
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,0.15,0.75,Jacobsonnich,...,0,22:52 PDT 11-07-2008,0,1,0,0,1,554,60,1
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,...,0,22:02 PDT 08-23-2008,0,0,0,0,0,131,69,1
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,...,0,14:23 PDT 08-24-2008,0,0,0,0,0,314,69,1


- P = cnt_n / cnt_uniq
- 输出的结果在`data_withn_cnt_n12`中，其中包含了所需要的3个值：cnt_n_2 cnt_uniq

In [59]:
# 2.2.3 计算P=cnt_n_2 / cnt_uniq: data_withn_cnt_n2

tmp = data_withn_cnt_n2['cnt_n_2'] / data_withn_cnt_n2['cnt_uniq']
data_withn_cnt_n2['P'] = tmp
data_withn_cnt_n2.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,...,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,id,n_2,cnt_uniq,cnt_n_2,P
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,0.15,0.75,Racer11,...,19:52 PDT 09-16-2008,0,0,0,0,0,89,69,1,0.014493
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,0.15,0.75,Cemo23,...,11:17 PDT 08-28-2008,0,0,0,0,1,498,60,1,0.016667
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,0.15,0.75,Jacobsonnich,...,22:52 PDT 11-07-2008,0,1,0,0,1,554,60,1,0.016667
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,...,22:02 PDT 08-23-2008,0,0,0,0,0,131,69,1,0.014493
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,...,14:23 PDT 08-24-2008,0,0,0,0,0,314,69,1,0.014493


In [60]:
# 2.2.4 逻辑检查
# data_withn_cnt_n1.to_csv("data_check_tmp.csv", header=True, encoding="utf-8",index=False)

## 2.3 保存结果
0. 逻辑检查请在之前进行
1. **这里保存的是没有经过`threshold`筛选的data**
2. 去重注意不能按照`unique_setting`去搞，毕竟一个`unique_setting`对应一组`np`值，也就是对应一个概率分布
3. 保存csv之前注意转换一些列的dtype

In [61]:
data_withn_cnt_n2.rename(columns={'n_2': 'N'}, inplace=True)
len_before_drop = data_withn_cnt_n2.shape[0]
data_withn_cnt_n2.drop_duplicates(subset=(['N','P']+unique_setting),inplace=True)
print("Drop 掉了 *{}* 行duplicate data".format(len_before_drop - data_withn_cnt_n2.shape[0]))

Drop 掉了 *38910* 行duplicate data


In [62]:
# output to csv
data_withn_cnt_n2.to_csv(output_path, header=True, encoding="utf-8",index=False)
data_withn_cnt_n2.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,...,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,id,N,cnt_uniq,cnt_n_2,P
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,0.15,0.75,Racer11,...,19:52 PDT 09-16-2008,0,0,0,0,0,89,69,1,0.014493
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,0.15,0.75,Cemo23,...,11:17 PDT 08-28-2008,0,0,0,0,1,498,60,1,0.016667
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,0.15,0.75,Jacobsonnich,...,22:52 PDT 11-07-2008,0,1,0,0,1,554,60,1,0.016667
3,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,...,22:02 PDT 08-23-2008,0,0,0,0,0,131,69,1,0.014493
4,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,...,14:23 PDT 08-24-2008,0,0,0,0,0,314,69,1,0.014493


# 3. 根据threshold筛选data
1. **取样本数在threshold之上的setting作为数据集来使用**
    - 根据每个unique setting下的样本数: data_withn_cnt_n2['cnt_uniq']，用`threshold`筛选
2. GTmodel可以根据这个进行计算

In [63]:
# 筛选unique setting对应的样本数(data_withn_cnt_n2['cnt_uniq'])在threshold之上的部分
data_withn_cnt_n2 = pd.read_csv(output_path,encoding="utf-8")
data_selected = data_withn_cnt_n2[data_withn_cnt_n2['cnt_uniq'] >= threshold].copy()
data_selected.to_csv(data_selected_path,header=True,encoding="utf-8",index=False)

In [64]:
b = len(data_withn_cnt_n2.groupby(unique_setting))
tmp = data_selected.groupby(unique_setting)
print(f"经过threshold，drop掉 *{b - len(tmp)}* 个setting")
print(f"现在有 *{len(tmp)}* 个uniq settings")
print(f"uniq settings = {unique_setting}")

经过threshold，drop掉 *2206* 个setting
现在有 *1196* 个uniq settings
uniq settings = ['id', 'bidincrement', 'bidfee', 'retail']
